In [1]:
import requests
import numpy as np
import pandas as pd
import pulp
import itertools

In [2]:

# Set your Mapbox API token
api_token = 'pk.eyJ1Ijoia3VuYWx1bWJyYW5pIiwiYSI6ImNsbm5lMnhkbzA1cngycm82eW5kaXJiejIifQ.xTo8-ITbtPQBKq2aqGNVww'

# Define the URL and necessary parameters
url = "https://api.mapbox.com/directions/v5/{profile}/{coordinates}"
profile = "mapbox/driving"  # Choose the appropriate profile based on your requirements

customer_count = 10
vehicle_count = 4

# the capacity of vehicle
vehicle_capacity = 50

# fix random seed
np.random.seed(seed=777)
depot_latitude = 40.748817
depot_longitude = -73.985428

# make dataframe which contains vending machine location and demand
df = pd.DataFrame({"latitude":np.random.normal(depot_latitude, 0.007, customer_count), 
                   "longitude":np.random.normal(depot_longitude, 0.007, customer_count), 
                   "demand":np.random.randint(10, 20, customer_count)})

# set the depot as the center and make demand 0 ('0' = depot)
df.iloc[0, df.columns.get_loc("latitude")] = depot_latitude
df.iloc[0, df.columns.get_loc("longitude")] = depot_longitude
df.iloc[0, df.columns.get_loc("demand")] = 0

coordinates_list = list(zip(df['longitude'], df['latitude']))

distances = {}

for i in range(len(coordinates_list)):
    for j in range(i + 1, len(coordinates_list)):
        coordinates = f"{coordinates_list[i][0]},{coordinates_list[i][1]};{coordinates_list[j][0]},{coordinates_list[j][1]}"
        response = requests.get(url.format(profile=profile, coordinates=coordinates), params={"access_token": api_token})
        data = response.json()
        distances[f"Distance_{i}_{j}"] = data


# Example of printing the distances
for key, value in distances.items():
    print(key, ":", value)

Distance_0_1 : {'routes': [{'weight_name': 'auto', 'weight': 1036.56, 'duration': 883.706, 'distance': 3137.797, 'legs': [{'via_waypoints': [], 'admins': [{'iso_3166_1_alpha3': 'USA', 'iso_3166_1': 'US'}], 'weight': 1036.56, 'duration': 883.706, 'steps': [], 'distance': 3137.797, 'summary': 'West 34th Street, FDR Drive'}], 'geometry': 'awuwF|gqbMdb@orAR\\|EfBtObBvJzFvAnAfCn@fD?PcAwGc@{NmIkVcD'}], 'waypoints': [{'distance': 0.858, 'name': 'West 34th Street', 'location': [-73.985433, 40.74881]}, {'distance': 2.527, 'name': 'FDR Drive', 'location': [-73.972134, 40.743049]}], 'code': 'Ok', 'uuid': 'l-nhfwoNPPJCq9vOzxG9gKzO2vQZwUQt9huIy2MSsfoaS-K7I_ZpLw=='}
Distance_0_2 : {'routes': [{'weight_name': 'auto', 'weight': 406.723, 'duration': 331.273, 'distance': 784.772, 'legs': [{'via_waypoints': [], 'admins': [{'iso_3166_1_alpha3': 'USA', 'iso_3166_1': 'US'}], 'weight': 406.723, 'duration': 331.273, 'steps': [], 'distance': 784.772, 'summary': '5th Avenue, West 31st Street'}], 'geometry': 'aw

In [3]:
n = len(df)
distance_result = np.zeros((n, n))

# Updating the 2D matrix with distances from the output
for key, value in distances.items():
    indices = key.split("_")[1:]
    i, j = int(indices[0]), int(indices[1])
    distance_result[i][j] = value['routes'][0]['distance']
    distance_result[j][i] = value['routes'][0]['distance']  # Assuming symmetric distances

# Printing the resulting 2D matrix
print(distance_result)

[[   0.    3137.797  784.772 1500.889 1546.382 1455.839 1188.937 1545.094
  2145.85  1464.598]
 [3137.797    0.    1924.357 2643.239 2374.498 2283.955 2493.982 2080.267
  1776.155 2280.737]
 [ 784.772 1924.357    0.     806.859 1639.219 1548.676 1058.173 1937.664
  2532.155 1562.992]
 [1500.889 2643.239  806.859    0.    2189.088 2098.545 1608.042 1130.805
  1725.295 2500.308]
 [1546.382 2374.498 1639.219 2189.088    0.     613.304 1143.042 2981.15
  3581.906 1015.067]
 [1455.839 2283.955 1548.676 2098.545  613.304    0.    1233.584 3071.693
  3672.449 1105.61 ]
 [1188.937 2493.982 1058.173 1608.042 1143.042 1233.584    0.    2937.875
  3538.63  1599.708]
 [1545.094 2080.267 1937.664 1130.805 2981.15  3071.693 2937.875    0.
  1312.246 2297.83 ]
 [2145.85  1776.155 2532.155 1725.295 3581.906 3672.449 3538.63  1312.246
     0.    3056.727]
 [1464.598 2280.737 1562.992 2500.308 1015.067 1105.61  1599.708 2297.83
  3056.727    0.   ]]


In [4]:
for vehicle_count in range(1,vehicle_count+1):
    
    # definition of LpProblem instance
    problem = pulp.LpProblem("CVRP", pulp.LpMinimize)
    #represents the routes taken by the vehicles
    x = [[[pulp.LpVariable("x%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(vehicle_count)]for j in range(customer_count)] for i in range(customer_count)]
    # add objective function
    problem += pulp.lpSum(distance_result[i][j] * x[i][j][k] if i != j else 0
                          for k in range(vehicle_count) 
                          for j in range(customer_count) 
                          for i in range (customer_count))
    # constraints
    # foluma (2)
    for j in range(1, customer_count):
        problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                              for i in range(customer_count) 
                              for k in range(vehicle_count)) == 1 

    # foluma (3)
    for k in range(vehicle_count):
        problem += pulp.lpSum(x[0][j][k] for j in range(1,customer_count)) == 1
        problem += pulp.lpSum(x[i][0][k] for i in range(1,customer_count)) == 1

    # foluma (4)
    for k in range(vehicle_count):
        for j in range(customer_count):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                                  for i in range(customer_count)) -  pulp.lpSum(x[j][i][k] for i in range(customer_count)) == 0

    #formula (5)
    for k in range(vehicle_count):
        problem += pulp.lpSum(df.demand[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) <= vehicle_capacity 


    # fomula (6)
    subtours = []
    for i in range(2,customer_count):
         subtours += itertools.combinations(range(1,customer_count), i)

    for s in subtours:
        problem += pulp.lpSum(x[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(vehicle_count)) <= len(s) - 1

    
    # print vehicle_count which needed for solving problem
    # print calculated minimum distance value
    if problem.solve() == 1:
        print('Vehicle Requirements:', vehicle_count)
        print('Moving Distance:', pulp.value(problem.objective))
        break

Vehicle Requirements: 3
Moving Distance: 16335.992000000002


In [5]:
import plotly.express as px
import json
import polyline
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", zoom=12)
color_list = ["red", "blue", "green"]
# # Adding the routes to the figure
# for k in range(vehicle_count):
#     for i in range(customer_count):
#         for j in range(customer_count):
#             if i != j and pulp.value(x[i][j][k]) == 1:
#                 fig.add_trace(px.line_mapbox(
#                     lat=[df.latitude[i], df.latitude[j]],
#                     lon=[df.longitude[i], df.longitude[j]]
#                 ).data[0])

# fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=depot_longitude, mapbox_center_lat=depot_latitude)
# fig.show()

# Adding the routes to the figure
for k in range(vehicle_count):
    for i in range(customer_count):
        for j in range(customer_count):
            if i != j and f"Distance_{i}_{j}" in distances:
                route_data = distances[f"Distance_{i}_{j}"]
                if 'routes' in route_data and route_data['routes']:
                    route_distance = route_data['routes'][0]['distance']
                    encoded_polyline = route_data['routes'][0]['geometry']
                    route_geometry = polyline.decode(encoded_polyline)
                    lon_list, lat_list = zip(*[(lon, lat) for lat, lon in route_geometry])
                    fig.add_trace(px.line_mapbox(
                        lat=lat_list,
                        lon=lon_list
                    ).data[0])
                else:
                    print(f"No route data found for points {i} and {j}.")

fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=depot_longitude, mapbox_center_lat=depot_latitude)
fig.show()

In [11]:
vehicle_routes = {k: [] for k in range(vehicle_count)}
import plotly.graph_objs as go
def get_user_input():
    while True:
        try:
            start_node = 1 
            end_node = 9
            if 0 < start_node <= 10 and 0 < end_node <= 10 and start_node != end_node:
                return start_node, end_node
            else:
                print("Invalid input. Please enter valid nodes.")
        except ValueError:
            print("Invalid input. Please enter valid nodes.")

# Getting user input for each vehicle's route
for k in range(vehicle_count):
    print(f"\nFor Vehicle {k + 1}:")
    start_node, end_node = get_user_input()
    if f"Distance_{start_node}_{end_node}" in distances:
        route_data = distances[f"Distance_{start_node}_{end_node}"]
        if 'routes' in route_data and route_data['routes']:
            route_distance = route_data['routes'][0]['distance']
            encoded_polyline = route_data['routes'][0]['geometry']
            route_geometry = polyline.decode(encoded_polyline)
            lon_list, lat_list = zip(*[(lon, lat) for lat, lon in route_geometry])
            vehicle_routes[k] = list(zip(lon_list, lat_list))
        else:
            print(f"No route data found for points {start_node} and {end_node}.")
    else:
        print(f"No distance data found for points {start_node} and {end_node}.")

# Plotting each vehicle's route
for k, route in vehicle_routes.items():
    if route:  # Checking if the route is not empty
        lon, lat = zip(*route)
        fig.add_trace(go.Scattermapbox(
            mode="lines",
            lon=lon,
            lat=lat,
            line=dict(width=2, color=color_list[k]),
            hoverinfo="text",
            hovertext=f"Vehicle {k + 1} Route",
        ))

fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=depot_longitude, mapbox_center_lat=depot_latitude)
fig.show()



For Vehicle 1:

For Vehicle 2:

For Vehicle 3:
